In [17]:
from keras import backend as K

In [18]:
# just to see the input format for input_shape
K.image_data_format()

'channels_last'

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Dense, \
    Flatten, Activation
from keras.regularizers import l2, l1
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
training_directory = '/home/brianjquinlan/AircraftTest/DataSubset/TrainImages'
test_directory = '/home/brianjquinlan/AircraftTest/DataSubset/TestImages'

# train_samples = 110336
# test_samples = 27638
train_samples = 33033
test_samples = 8254

In [3]:
# tried to have to conv layers per each pooling but will take to long to run as of now
# so far not looking like a very good model, it is not yet finished for me
# would like to have more inputs on the first dense layer but also would take too long 

model = Sequential()
model.add(Convolution2D(32, (5, 5), activation='relu', input_shape=(405,270, 3)))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Activation('relu'))

model.add(Convolution2D(32, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Activation('relu'))

model.add(Convolution2D(32, (5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(69, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(69, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics=['accuracy'])

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 401, 266, 32)      2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 133, 88, 32)       0         
_________________________________________________________________
activation_1 (Activation)    (None, 133, 88, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 129, 84, 32)       25632     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 43, 28, 32)        0         
_________________________________________________________________
activation_2 (Activation)    (None, 43, 28, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 39, 24, 32)        25632     
__________

In [ ]:
# would like to use more epochs but will take to long.

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    training_directory,
    target_size=(405, 270),
    batch_size= 150,
    class_mode='categorical')

test_generator = datagen.flow_from_directory(
    test_directory,
    target_size=(405, 270),
    batch_size=150,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch = train_samples // 150,
    epochs = 10,
    validation_data = test_generator,
    validation_steps = train_samples // 150)

model.save_weights('starting_point.h5')

Found 33033 images belonging to 69 classes.
Found 8254 images belonging to 69 classes.
Epoch 1/10
 25/220 [==>...........................] - ETA: 1:00:04 - loss: 4.2344 - acc: 0.0176